In [4]:
#%pip install pyspark

In [5]:
#%pip install google-cloud-storage

TO DO:

- clean up code
- might want to not point to the latest gsc connector but an actual version for compatability
- write post on it

In [ ]:
from pyspark.sql import SparkSession
import os

# .config("spark.hadoop.gooogle.cloud.auth.null.credential.enabled", "true") \
# .config("spark.hadoop.google.cloud.auth.service.account.enable","true") \
#.config("spark.hadoop.google.cloud.auth.type","APPLICATION_DEFAULT") \
#.config("spark.jars.packages","com.google.cloud.bigdataoss:gcs-connector:hadoop3-2.2.26") \

"""
This works for spark to write directly to gcs in parquet; if i try to go via ivy with spark.jars.packages, it blows up and doesnt work
"""

spark = SparkSession.builder \
    .appName("GCSTest") \
    .config("spark.jars", "https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar") \
    .config("spark.hadoop.google.cloud.auth.service.account.enable","true") \
    .getOrCreate()


gcs_path = f"gs://{os.getenv("GCS_BUCKET")}/data_output2"

# Create a DataFrame with dummy data
data = [("Alice", 25), ("Bob", 30), ("Cathy", 35)]
columns = ["Name", "Age"]

df = spark.createDataFrame(data, columns)
#print(df)
df.write.mode("overwrite").parquet(gcs_path)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/27 06:24:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [1]:
## This works using a google service account
from pyspark.sql import SparkSession
import os
from pyspark.sql.functions import current_date, rand, floor, expr

catalog_name = "iceberg"

def get_spark_instance_via_gsa(catalog_name) -> SparkSession:

    spark_version = "3.5"
    scala_version = "2.12"
    iceberg_version = "1.7.0"

    warehouse_path = f"gs://{os.getenv("GCS_BUCKET")}/icehouse"
    #print(gcs_path)

    return SparkSession.builder \
        .appName("local_iceberg_example") \
        .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
        .config(f"spark.sql.catalog.{catalog_name}", "org.apache.iceberg.spark.SparkCatalog") \
        .config(f"spark.sql.catalog.{catalog_name}.type", "hadoop") \
        .config(f"spark.sql.catalog.{catalog_name}.warehouse", warehouse_path) \
        .config("spark.sql.warehouse.dir", warehouse_path) \
        .config("spark.jars.packages", f"org.apache.iceberg:iceberg-spark-runtime-{spark_version}_{scala_version}:{iceberg_version},com.google.guava:guava:30.1.1-jre") \
        .config("spark.jars", "https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar") \
        .config("spark.hadoop.google.cloud.auth.service.account.enable","true") \
        .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
        .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS") \
        .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", os.getenv("GOOGLE_APPLICATION_CREDENTIALS")) \
        .config("spark.driver.host", "localhost") \
        .config("spark.driver.bindAddress", "127.0.0.1") \
        .getOrCreate()

In [1]:
from pyspark.sql import SparkSession
import os

catalog_name = "iceberg"

#https://github.com/GoogleCloudDataproc/hadoop-connectors/blob/master/gcs/INSTALL.md
# adc seems to work now


def get_spark_instance_via_adc(catalog_name: str, gcs_bucket: str, app_name: str = "local_iceberg_example") -> SparkSession:
    """
    Creates a Spark session configured for reading and writing Iceberg tables on GCS with ADC.

    Args:
        catalog_name (str): The Iceberg catalog name.
        gcs_bucket (str): The GCS bucket to use as the Iceberg warehouse path.
        app_name (str): The name of the Spark application. Defaults to 'local_iceberg_example'.

    Returns:
        SparkSession: Configured Spark session instance.
    """
    # Versions for dependencies
    spark_version = os.getenv("SPARK_VERSION", "3.5")
    scala_version = os.getenv("SCALA_VERSION", "2.12")
    iceberg_version = os.getenv("ICEBERG_VERSION", "1.7.0")

    # Define the Iceberg warehouse path
    warehouse_path = f"gs://{gcs_bucket}/icehouse"

    jars = [
        f"org.apache.iceberg:iceberg-spark-runtime-{spark_version}_{scala_version}:{iceberg_version}",
        #"com.google.cloud.bigdataoss:gcs-connector:hadoop3-2.2.5",
        #"com.google.cloud.bigdataoss:gcs-connector:hadoop3-2.2.5",
        #"com.google.guava:guava:30.1.1-jre"
    ]

    # this jar here: https://github.com/GoogleCloudDataproc/hadoop-connectors/releases
    ## seems to be the one that works well (not the gcs-conenctor-hadoop3-latest, which is missing stuff)
    local_jar_path = "./jars/gcs-connector-3.0.4-shaded.jar"
    #local_jar_path = "./jars/gcs-connector-hadoop3-latest.jar"

    return SparkSession.builder \
        .appName(app_name) \
        .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
        .config(f"spark.sql.catalog.{catalog_name}", "org.apache.iceberg.spark.SparkCatalog") \
        .config(f"spark.sql.catalog.{catalog_name}.type", "hadoop") \
        .config(f"spark.sql.catalog.{catalog_name}.warehouse", warehouse_path) \
        .config("spark.jars.packages", ",".join(jars)) \
        .config("spark.jars", local_jar_path) \
        .config("spark.hadoop.google.cloud.auth.service.account.enable", "false") \
        .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
        .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS") \
        .config("spark.driver.host", "localhost") \
        .config("spark.driver.bindAddress", "127.0.0.1") \
        .config("spark.hadoop.google.cloud.auth.type", "APPLICATION_DEFAULT") \
        .getOrCreate()

In [2]:
spark = get_spark_instance_via_adc(catalog_name=catalog_name, gcs_bucket = os.getenv("GCS_BUCKET"))

:: loading settings :: url = jar:file:/Users/matthewmartin/dream_machine/iceberg/iceberg_gcs/.venv/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/matthewmartin/.ivy2/cache
The jars for the packages stored in: /Users/matthewmartin/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a8cf73f5-3a25-4ba2-8bed-662ab9dd3945;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.7.0 in central
:: resolution report :: resolve 42ms :: artifacts dl 1ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.7.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :

In [5]:
print(spark.conf.get(f"spark.sql.catalog.{catalog_name}.warehouse"))

gs://data-mattm-test-sbx/icehouse


In [8]:
spark.stop()

In [7]:
spark = get_spark_instance_via_gsa(catalog_name)

In [3]:
# Create a DataFrame with dummy data
data = [("Alice", 25), ("Bob", 30), ("Cathy", 35)]
columns = ["Name", "Age"]

df = spark.createDataFrame(data, columns)
print(df)


DataFrame[Name: string, Age: bigint]


In [4]:
namespace = "dummy_ns"

## not needed
#spark.sql(f"create namespace {namespace}")

In [6]:
table_name = "dummy_data7"
catalog_name = "iceberg"

df.writeTo(f"{catalog_name}.{namespace}.{table_name}") \
    .using("iceberg") \
    .createOrReplace()

24/12/27 10:09:42 WARN HadoopTableOperations: Error reading version hint file gs://data-mattm-test-sbx/icehouse/dummy_ns/dummy_data7/metadata/version-hint.text
java.io.FileNotFoundException: Item not found: 'gs://data-mattm-test-sbx/icehouse/dummy_ns/dummy_data7/metadata/version-hint.text'. Note, it is possible that the live version is still available but the requested generation is deleted.
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageExceptions.createFileNotFoundException(GoogleCloudStorageExceptions.java:47)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageImpl.open(GoogleCloudStorageImpl.java:678)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageImpl.open(GoogleCloudStorageImpl.java:669)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageFileSystemImpl.open(GoogleCloudStorageFileSystemImpl.java:309)
	at com.google.

In [7]:
from google.cloud import storage

client = storage.Client()
buckets = list(client.list_buckets())
print("Buckets available:", [bucket.name for bucket in buckets])

/Users/matthewmartin/dream_machine/iceberg/iceberg_gcs/.venv/lib/python3.13/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Buckets available: ['data-mattm-test-sbx']
